# 

# install packages

In [5]:
#pip install haystack-ai

In [6]:
# pip install mistralai

In [7]:
# pip install --upgrade mistralai

In [8]:
# so i can use .env-file where i stored my API-key
# pip install python-dotenv 

In [3]:
#pip install "haystack-ai[mistral]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install haystack-ai-mistral

ERROR: Could not find a version that satisfies the requirement haystack-ai-mistral (from versions: none)
ERROR: No matching distribution found for haystack-ai-mistral
Note: you may need to restart the kernel to use updated packages.


In [3]:
from haystack_mistral.components.generators import MistralChatGenerator


ModuleNotFoundError: No module named 'haystack_mistral'

In [4]:
pip install "git+https://github.com/deepset-ai/haystack-core-integrations.git#egg=haystack-integrations-mistral&subdirectory=integrations/mistral"

  Cloning https://github.com/deepset-ai/haystack-core-integrations.git to /private/var/folders/9l/ng7prn7d7z1gpmzs8nj4klgm0000gn/T/pip-install-_eiwj8nj/haystack-integrations-mistral_fdb64ddd2eb74f37860a8817aaa892f1
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack-core-integrations.git /private/var/folders/9l/ng7prn7d7z1gpmzs8nj4klgm0000gn/T/pip-install-_eiwj8nj/haystack-integrations-mistral_fdb64ddd2eb74f37860a8817aaa892f1
  Resolved https://github.com/deepset-ai/haystack-core-integrations.git to commit 4b003f2a4cf75d85fca6436fb91a4fe6d46e6006
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Discarding git+https://github.com/deepset-ai/haystack-core-integrations.git#egg=haystack-integrations-mistral&subdirectory=integrations/mistral: Requested mistral-haystack from git+https://github.com/deepset-ai/haystack-core-integrations.git#egg=haystack-integrations-mi

# Load API-key

In [9]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads the varialbe from .env-file
api_key = os.getenv("MISTRAL_API_KEY")

# Example to see if LLM is working


In [9]:
import os
from mistralai import Mistral

model = "mistral-large-latest"

client = Mistral(api_key=api_key)

#chat_response = client.chat.complete(
#    model= model, messages = [{
#            "role": "user",
#            "content": "What is the best French meal? Return the name and the ingredients in short JSON object.",
#        },])
#print(chat_response.choices[0].message.content)

Choosing the "best" French meal can be subjective, as it depends on personal preferences. However, one of the most iconic and beloved French dishes is "Coq au Vin." Here is a short JSON object with the name and main ingredients:

```json
{
  "name": "Coq au Vin",
  "ingredients": [
    "Chicken",
    "Red wine",
    "Bacon",
    "Onions",
    "Carrots",
    "Garlic",
    "Mushrooms",
    "Thyme",
    "Bay leaves",
    "Flour",
    "Butter",
    "Salt",
    "Pepper"
  ]
}
```

This dish is a classic French stew where chicken is slow-cooked in red wine with an assortment of vegetables and herbs.


In [1]:
from haystack import Pipeline, Document
from haystack.utils import Secret
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.prompt_builder import PromptBuilder

#from haystack.components.builders import DynamicChatPromptBuilder
from haystack.components.converters import TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.writers import DocumentWriter
#from haystack_integrations.components.embedders.mistral import MistralDocumentEmbedder, MistralTextEmbedder
from haystack_integrations.components.generators.mistral import MistralChatGenerator



# Write documents to InMemoryDocumentStore
document_store = InMemoryDocumentStore()
document_store.write_documents([
    Document(content="My name is Jean and I live in Paris."), 
    Document(content="My name is Mark and I live in Berlin."), 
    Document(content="My name is Giorgio and I live in Rome.")
])

# 3. Write a Prompt Template
prompt_template = """
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
"""

# 4. Create the components
retriever = InMemoryBM25Retriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt_template)
llm = MistralChatGenerator(api_key=Secret.from_token(api_key), 
                           model='mistral-large-latest')

# 5. Create a pipeline instance
rag_pipeline = Pipeline()

# 6. Add components to the pipeline
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)

# 7. Connect individual components
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

# 8. Ask a question and define the results
question = "Who lives in Paris?"
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

print(results["llm"]["replies"])

ModuleNotFoundError: No module named 'haystack_integrations'

In [5]:
from haystack import Pipeline, Document
from haystack.utils import Secret
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.prompt_builder import PromptBuilder

# Write documents to InMemoryDocumentStore
document_store = InMemoryDocumentStore()
document_store.write_documents([
    Document(content="My name is Jean and I live in Paris."), 
    Document(content="My name is Mark and I live in Berlin."), 
    Document(content="My name is Giorgio and I live in Rome.")
])

# Build a RAG pipeline
prompt_template = """
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
"""

retriever = InMemoryBM25Retriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt_template)
#llm = OpenAIGenerator(api_key=Secret.from_token(api_key))
llm = MistralChatGenerator(api_key=Secret.from_token(api_key), 
                           model='mistral-large-latest')

rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

# Ask a question
question = "Who lives in Paris?"
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

print(results["llm"]["replies"])

NameError: name 'MistralChatGenerator' is not defined

In [7]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.dataclasses import ChatMessage
from haystack.utils.auth import Secret

from haystack.components.builders import DynamicChatPromptBuilder
from haystack.components.converters import TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.writers import DocumentWriter
from haystack_integrations.components.embedders.mistral import MistralDocumentEmbedder, MistralTextEmbedder
from haystack_integrations.components.generators.mistral import MistralChatGenerator

document_store = InMemoryDocumentStore()

docs = TextFileToDocument().run(sources=["essay.txt"])
split_docs = DocumentSplitter(split_by="passage", split_length=2).run(documents=docs["documents"])
embeddings = MistralDocumentEmbedder(api_key=Secret.from_token(api_key)).run(documents=split_docs["documents"])
DocumentWriter(document_store=document_store).run(documents=embeddings["documents"])


text_embedder = MistralTextEmbedder(api_key=Secret.from_token(api_key))
retriever = InMemoryEmbeddingRetriever(document_store=document_store)
prompt_builder = DynamicChatPromptBuilder(runtime_variables=["documents"])
llm = MistralChatGenerator(api_key=Secret.from_token(api_key), 
                           model='mistral-large-latest')

chat_template = """Answer the following question based on the contents of the documents.\n
                Question: {{query}}\n
                Documents: 
                {% for document in documents %}
                    {{document.content}}
                {% endfor%}
                """
messages = [ChatMessage.from_user(chat_template)]

rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", text_embedder)
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)


rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder.prompt", "llm.messages")

question = "What were the two main things the author worked on before college?"

result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"template_variables": {"query": question}, "prompt_source": messages},
        "llm": {"generation_kwargs": {"max_tokens": 225}},
    }
)

print(result["llm"]["replies"][0].content)

ImportError: cannot import name 'DynamicChatPromptBuilder' from 'haystack.components.builders' (/opt/homebrew/Caskroom/miniforge/base/envs/elisabeth_python-umgebung_nr1/lib/python3.11/site-packages/haystack/components/builders/__init__.py)

# Example RAG

In [10]:
from mistralai import Mistral
import requests
import numpy as np
#import faiss
import os
from getpass import getpass

#api_key= getpass("Type your API Key")
client = Mistral(api_key=api_key)

In [13]:
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

f = open('essay.txt', 'w')
f.write(text)
f.close()

In [15]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
print(len(chunks))

def get_text_embedding(input):
    embeddings_batch_response = client.embeddings.create(
          model="mistral-embed",
          inputs=input
      )
    return embeddings_batch_response.data[0].embedding
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])



37


SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}

# 1. Create a Pipeline
## 1.1. Import dependencies

In [3]:
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

## 1.2. Initialize components

In [ ]:
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
text_embedder = SentenceTransformersTextEmbedder()
retriever = InMemoryEmbeddingRetriever(document_store=document_store)

## 1.3. Create the pipeline

In [ ]:
query_pipeline = Pipeline()

## 1.4. Add components

In [ ]:

# Here is an example of how you'd add the components initialized in step 2 above:
query_pipeline.add_component("text_embedder", text_embedder)
query_pipeline.add_component("retriever", retriever)

rag_pipeline.add_component("llm", llm)

## 1.5. Connect components

## 1.6. Run the pipeline
